# Data Aggregation and Group Operations

## GroupBy mechanics

In [4]:
df = DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                'key2' : ['one', 'two', 'one', 'two', 'one'],
                'data1' : np.random.randn(5),
                'data2' : np.random.randn(5)})
df

      data1     data2 key1 key2
0 -0.204708  1.393406    a  one
1  0.478943  0.092908    a  two
2 -0.519439  0.281746    b  one
3 -0.555730  0.769023    b  two
4  1.965781  1.246435    a  one

In [8]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005    0.478943
            2006   -0.519439
Ohio        2005   -0.380219
            2006    1.965781
Name: data1, dtype: float64

### Iterating over groups

### Selecting a column or subset of columns

## Data aggregation

### Column-wise and multiple function application

## Group-wise operations and transformations

### Apply: General split-apply-combine

#### Suppressing the group keys

### Quantile and bucket analysis

### Example: Filling missing values with group-specific values

In [55]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4
data = Series(np.random.randn(8), index=states)
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio         -0.607638
New York     -0.195258
Vermont            NaN
Florida       0.422648
Oregon        2.001614
Nevada             NaN
California   -1.104685
Idaho              NaN
dtype: float64

## Pivot tables and Cross-tabulation